In [1]:
import pandas as pd
import numpy as np
import statsmodels.regression.linear_model as sm
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tools.tools import add_constant

### Linear Regression

In [32]:
returns = pd.read_csv("/Users/luzhang/Documents/GitHub/WrappingUp_Exploring_Intraday_Momentum_2rd_try/Data/SPY_Returns_with_regime.csv")

In [33]:
returns = add_constant(returns)
returns

,const,datetime,R_LH,R_LH0,R_ROD,R_ON,R_FH,R_ONFH,R_M,R_SLH,regime
0,1.0,2007-06-15,-0.000783,-0.000785,0.003012,0.003471,-0.000065,0.003405,0.000131,-0.000522,0
1,1.0,2007-06-18,-0.000458,-0.000783,-0.000523,0.002025,-0.001043,0.000980,-0.000914,-0.000588,0
2,1.0,2007-07-10,-0.003500,-0.000914,-0.010843,-0.005422,-0.001511,-0.006924,-0.003420,-0.000528,0
3,1.0,2007-08-01,0.014601,-0.009672,-0.006121,-0.001307,0.000482,-0.000825,-0.005438,0.000138,1
4,1.0,2007-08-02,-0.002303,0.014601,0.006889,0.001023,0.003339,0.004365,-0.006180,0.008746,0
...,...,...,...,...,...,...,...,...,...,...,...
2768,1.0,2018-10-25,-0.004533,-0.007299,0.023188,0.008106,0.002618,0.010746,0.011975,0.000331,0
2769,1.0,2018-10-26,0.000189,-0.004533,-0.018139,-0.015622,0.000075,-0.015547,-0.005528,0.002911,1
2770,1.0,2018-10-29,0.004991,0.000189,-0.010554,0.013155,0.000781,0.013947,-0.016543,-0.007749,0
2771,1.0,2018-10-30,0.002639,0.004991,0.012528,-0.000493,0.012250,0.011751,-0.002286,0.003061,0


In [34]:
y = returns.R_SLH

In [35]:
X = returns[["R_LH0","R_ON","R_FH","R_ONFH","R_M"]]

In [36]:
# define a function find the features after backward Elimination
def BackwardEli(X,y):
    cols = list(X.columns)
    pmax = 1
    while (len(cols)>0):
        p = []
        X_1 = X[cols]
        X_1 = add_constant(X_1)
        model = sm.OLS(y,X_1).fit()
        p = pd.Series(model.pvalues.values[1:],index=cols)
        pmax = max(p)
        feature_with_p_max = p.idxmax()
        if(pmax>0.05):
            cols.remove(feature_with_p_max)
        else:
            break
    return cols

In [37]:
cols = BackwardEli(X,y)
final_input_X = add_constant(returns[cols])
final = sm.OLS(y,final_input_X).fit()
final.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  R_SLH   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     38.57
Date:                Thu, 12 Oct 2023   Prob (F-statistic):           1.86e-31
Time:                        14:57:58   Log-Likelihood:                 12088.
No. Observations:                2773   AIC:                        -2.417e+04
Df Residuals:                    2768   BIC:                        -2.414e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.191e-05    5.9e-05      0.372      0.710   -9.37e-05       0.000
R_LH0         -0.1084      0.015     -7.034      0.000      -0.139      -0.078
R_ON           8.1774      0.770     10.615      0.000       6.667       9.688
R_FH           8.1347      0.764     10.641      0.000       6.636       9.634
R_ONFH        -8.1588      0.769    -10.611      0.000      -9.667      -6.651
==============================================================================
Omnibus:                     2055.332   Durbin-Watson:                   2.188
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           276167.146
Skew:                           2.665   Prob(JB):                         0.00
Kurtosis:                      51.598   Cond. No.                     2.26e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.26e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""